# Experimenting with the Co:here API

## TLDR summarys

In [4]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [5]:
import cohere
api_key = ''
co = cohere.Client(api_key)

In [4]:
prompt = f"""Passage: Is Wordle getting tougher to solve? Players seem to be convinced that the game has gotten harder in recent weeks ever since The New York Times bought it from developer Josh Wardle in late January. The Times has come forward and shared that this likely isn't the case. That said, the NYT did mess with the back end code a bit, removing some offensive and sexual language, as well as some obscure words There is a viral thread claiming that a confirmation bias was at play. One Twitter user went so far as to claim the game has gone to "the dusty section of the dictionary" to find its latest words.

TLDR: Wordle has not gotten more difficult to solve.
--
Passage: ArtificialIvan, a seven-year-old, London-based payment and expense management software company, has raised $190 million in Series C funding led by ARG Global, with participation from D9 Capital Group and Boulder Capital. Earlier backers also joined the round, including Hilton Group, Roxanne Capital, Paved Roads Ventures, Brook Partners, and Plato Capital.

TLDR: ArtificialIvan has raised $190 million in Series C funding.
--
Passage: The National Weather Service announced Tuesday that a freeze warning is in effect for the Bay Area, with freezing temperatures expected in these areas overnight. Temperatures could fall into the mid-20s to low 30s in some areas. In anticipation of the hard freeze, the weather service warns people to take action now.

TLDR:"""

In [5]:
response = co.generate( 
    model='xlarge', 
    prompt = prompt,
    max_tokens=40, 
    temperature=0.8,
    stop_sequences=["--"])

summary = response.generations[0].text

In [6]:
summary

' A freeze warning is in effect for the Bay Area.\n--'

## Creating tags from a large text

Donwloading short story examples from [reedsy](https://blog.reedsy.com/short-stories/)

In [2]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

c:\Users\bensn\anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
from scrape_functions import *

In [42]:
url_list = ['https://blog.reedsy.com/short-story/ndit0a/', 
            'https://blog.reedsy.com/short-story/n5toc8/', 
            'https://blog.reedsy.com/short-story/eoex6z/', 
            'https://blog.reedsy.com/short-story/vi8uge/', 
            'https://blog.reedsy.com/short-story/7yrery/',
            'https://blog.reedsy.com/short-story/j48lmh/',
            'https://blog.reedsy.com/short-story/1toaoe/',
            'https://blog.reedsy.com/short-story/7yrery/',
            'https://blog.reedsy.com/short-story/c2ar53/',
            'https://blog.reedsy.com/short-story/iur9u1/',
            'https://blog.reedsy.com/short-story/0wjsaw/',
            'https://blog.reedsy.com/short-story/rcuiva/',
            'https://blog.reedsy.com/short-story/zf6eb9/',
            'https://blog.reedsy.com/short-story/evwarr/',
            'https://blog.reedsy.com/short-story/omxn2o/',
            'https://blog.reedsy.com/short-story/gm80ij/',
            'https://blog.reedsy.com/short-story/dkhusv/',
            'https://blog.reedsy.com/short-story/idca55/',
            'https://blog.reedsy.com/short-story/ua19kk/',
            'https://blog.reedsy.com/short-story/vdgjfo/',
            'https://blog.reedsy.com/short-story/5antpt/',
            'https://blog.reedsy.com/short-story/4gdaax/',
            'https://blog.reedsy.com/short-story/nkkeif/',
            'https://blog.reedsy.com/short-story/nc54kv/']

In [43]:
scrape_URL('https://blog.reedsy.com/short-story/ndit0a/', driver)

,Article title,Body,Tags
0,Mothering,"Carmen invades my mind a lot, even years after...","Fiction,Teens & Young Adult,Drama"


In [44]:
df = pd.DataFrame()
for url in url_list:
    df = df.append(scrape_URL(url, driver))

In [46]:
df

,Article title,Body,Tags
0,Mothering,"Carmen invades my mind a lot, even years after...","Fiction,Teens & Young Adult,Drama"
0,10 Ways to Explain Your Husband's Deat...,Content warning: Themes of death***1. Tell the...,"Christian,Drama,Sad"
0,"And the Radio Said, “There’s Another S...","CW: Gun violence, deathThe air hung heavy with...","Drama,Crime,Historical Fiction"
0,Long Distance Phone Call,You can talk to the dead on the pay phone in f...,"Fiction,Lesbian"
0,Wish You Were Here,Bonjour from the city of love! I don't have mu...,"Contemporary,Funny,Sad"
0,Old Soul,Can you hear me?I remember being reborn a thou...,Fiction
0,Your Cat™️ Customer Service,"Hi, I'm Timmy McHill and drugs and AA started ...","Crime,Fiction"
0,Wish You Were Here,Bonjour from the city of love! I don't have mu...,"Contemporary,Funny,Sad"
0,We/I,Something happened in the lab the other night....,Science Fiction
0,Someone Has Died,TW: This story contains several depictions and...,"Fiction,Horror,Speculative"


In [12]:
df.to_csv('reedsy.csv', index=False, encoding='utf-8')

In [19]:
summaries = ["Carmen as a woman who slept with men for money did not resonate with the Carmen of my childhood. It's been a long time since Carmen died, and just as long since I've seen Tally or her sisters.", "After you've tucked him and Waffles into bed and he asks you again where his father is, tell him your husband will be staying with God for some time. What about those times your husband stumbled into bed after midnight and told you he was out doing some last-minute repairs for a client, and they'd invited him inside for some drinks?"]

In [41]:
import cohere
co = cohere.Client(api_key)
response = co.generate(
  model='large',
  prompt=f"Passage: {df['Body'].values[0][:2000]}\n\nTLDR:{summaries[0]}\n--\nPassage:{df['Body'].values[1][:2000]}\n\nTLDR:{summaries[1]}\n--\nPassage:{df['Body'].values[2][:2000]}\n\nTLDR:",
  max_tokens=15,
  temperature=0.8,
  k=0,
  p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop_sequences=["--"],
  return_likelihoods='NONE')
print('Prediction: {}'.format(response.generations[0].text))

Prediction: It's a hard life being Ma's child.
--


In [50]:
import cohere
from cohere.classify import Example
co = cohere.Client(api_key)
response = co.classify(
  model='large',
  inputs=["Any decent trader spends 90% of their efforts exploring how they could be wrong. That should apply to everyone’s decision making.", "Congratulations to all of this years @OliverAwards winners!"],
  examples=[Example("Elon Musk says Twitter Blue subscribers should be able to pay with dogecoin", "Business news"), Example("Probability of a US recession in the next 12 months, via WSJ", "Business news"), Example("European futures slide", "Business news"), Example("NASDAQ rises 2% to ATH", "Business news"), Example("FTX Founder is one of the world\'s richest crypto billionaires, with a fortune valued at $20 billion.", "Business news"), Example("Sweet Potato Macaroni Cheese is #RecipeOfTheDay, and I’m very happy about it!", "Cooking"), Example("3-Ingredient Slow Cooker recipes", "Cooking"), Example("This is by far the BEST biscuit recipe I’ve ever tried", "Cooking"), Example("Baking my first loaf of banana bread...", "Cooking"), Example("From the queen of Italian cooking, this is one of the most iconic tomato sauce recipes ever", "Cooking"), Example("I’ve actually read this book and it was extremely insightful. A quick read as well and available as a free audiobook through many libraries.", "Arts & Culture"), Example("Today’s Daily Cartoon", "Arts & Culture"), Example("Get a glimpse of the stage adaptation of Hayao Miyazaki’s 2001 Oscar-winning animated feature Spirited Away", "Arts & Culture"), Example("The #Banksy Exhibit in Cambridge, MA is absolutely terrific.", "Arts & Culture"), Example("“A Whisper in Time” large abstract paining 48’ x 48’", "Arts & Culture")])
print('The confidence levels of the labels are: {}'.format(response.classifications))

The confidence levels of the labels are: [cohere.Classification {
	input: Any decent trader spends 90% of their efforts exploring how they could be wrong. That should apply to everyone’s decision making.
	prediction: Business news
	confidence: [cohere.Confidence {
	label: Business news
	confidence: 0.89163196
}, cohere.Confidence {
	label: Cooking
	confidence: 0.040212233
}, cohere.Confidence {
	label: Arts & Culture
	confidence: 0.068155825
}]
	labels: {'Arts & Culture': cohere.LabelPrediction {
	confidence: 0.068155825
}, 'Business news': cohere.LabelPrediction {
	confidence: 0.89163196
}, 'Cooking': cohere.LabelPrediction {
	confidence: 0.040212233
}}
}, cohere.Classification {
	input: Congratulations to all of this years @OliverAwards winners!
	prediction: Arts & Culture
	confidence: [cohere.Confidence {
	label: Business news
	confidence: 0.038718157
}, cohere.Confidence {
	label: Cooking
	confidence: 0.2552241
}, cohere.Confidence {
	label: Arts & Culture
	confidence: 0.7060577
}]